# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
city_csv = "../starter_code/city_data.csv"
city_data_df = pd.read_csv('cities.csv')
city_data


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
city_data_df.dtypes

City_ID         int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [12]:
#api key
gmaps.configure(api_key=g_key)

city_data = city_data_df.dropna()
humidity = city_data_df["Humidity"].astype(float)

locations = city_data_df[["Lat", "Lng"]].astype(float)


# Create a Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_data_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
humidity = city_data_df["Humidity"].astype(float)
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
humidity_float=humidity.astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_float, dissipating=False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
#Narrow down the cities to fit weather conditions.
Temperature = []


clean_data =  city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70)]
#clean_data = city_data_df.loc[clean_data["Wind Speed"] < 10]
#clean_data = city_data_df.loc[clean_data["Cloudiness"] == 0]


clean_data = city_data_df.dropna()

clean_data





,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = city_data_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,ostrovnoy,RU,37.50,68.05,39.51,
1,mahebourg,MU,78.80,-20.41,57.70,
2,qaanaaq,GL,22.20,77.48,-69.36,
3,zhuhai,CN,44.60,40.71,112.04,
4,cape town,ZA,55.99,-33.93,18.42,
...,...,...,...,...,...,...
543,yabrud,SY,80.60,33.97,36.66,
544,paraiso,MX,84.20,24.01,-104.61,
545,veraval,FR,61.00,49.65,0.71,
546,novyy urgal,RU,49.74,51.07,132.56,


In [44]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,ostrovnoy,RU,37.50,68.05,39.51,
1,mahebourg,MU,78.80,-20.41,57.70,
2,qaanaaq,GL,22.20,77.48,-69.36,
3,zhuhai,CN,44.60,40.71,112.04,
4,cape town,ZA,55.99,-33.93,18.42,
...,...,...,...,...,...,...
543,yabrud,SY,80.60,33.97,36.66,
544,paraiso,MX,84.20,24.01,-104.61,
545,veraval,FR,61.00,49.65,0.71,
546,novyy urgal,RU,49.74,51.07,132.56,


In [55]:
hotel_df = clean_data

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    my_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Skip.")
    
    print("-----------------------------")


Results for Index 0: ostrovnoy.
Skip.
-----------------------------
Results for Index 1: mahebourg.
Closest hotel is Shandrani Beachcomber Resort & Spa.
-----------------------------
Results for Index 2: qaanaaq.
Closest hotel is Qaanaaq Hotel.
-----------------------------
Results for Index 3: zhuhai.
Skip.
-----------------------------
Results for Index 4: cape town.
Closest hotel is Southern Sun Waterfront Cape Town.
-----------------------------
Results for Index 5: atuona.
Closest hotel is Hôtel Hiva Oa Hanakee Lodge.
-----------------------------
Results for Index 6: flinders.
Closest hotel is Shellharbour Resort & Conference Centre.
-----------------------------
Results for Index 8: tiksi.
Closest hotel is Umka.
-----------------------------
Results for Index 9: pushkinskiye gory.
Closest hotel is The recreation center "Pushkinogorye".
-----------------------------
Results for Index 10: aykhal.
Closest hotel is Hotel Astoria.
-----------------------------
Results for Index 11: c

Closest hotel is Hôtel & Restaurant DIANA Ambilobe.
-----------------------------
Results for Index 88: cidreira.
Closest hotel is Hotel Castelo.
-----------------------------
Results for Index 89: ternate.
Closest hotel is Puerto Azul Clubhouse.
-----------------------------
Results for Index 90: tarauaca.
Closest hotel is Flat Temporada.
-----------------------------
Results for Index 91: bella vista.
Closest hotel is Hotel Río Arriba.
-----------------------------
Results for Index 92: xichang.
Closest hotel is Xichang ibis Hotel.
-----------------------------
Results for Index 93: antioch.
Closest hotel is Days Inn & Suites by Wyndham Antioch.
-----------------------------
Results for Index 94: najran.
Closest hotel is Daniel Hotel Apartments 1 Najran.
-----------------------------
Results for Index 95: agirish.
Skip.
-----------------------------
Results for Index 96: mataura.
Closest hotel is Ellie's Villa.
-----------------------------
Results for Index 97: katsuura.
Closest hot

Closest hotel is Teya la bella.
-----------------------------
Results for Index 171: vitim.
Closest hotel is Uyut.
-----------------------------
Results for Index 172: antsohihy.
Closest hotel is Hotel de Ville.
-----------------------------
Results for Index 173: kavieng.
Closest hotel is Nusa Island Retreat.
-----------------------------
Results for Index 174: la esperanza.
Closest hotel is Hotel Ipsan-Nah.
-----------------------------
Results for Index 175: clyde river.
Closest hotel is Qamaq Hotel.
-----------------------------
Results for Index 176: nador.
Closest hotel is Hôtel La Giralda.
-----------------------------
Results for Index 177: keflavik.
Closest hotel is Hotel Keflavik.
-----------------------------
Results for Index 178: faya.
Closest hotel is أبو تركي للوحدات السكنيه.
-----------------------------
Results for Index 179: chokurdakh.
Skip.
-----------------------------
Results for Index 180: le marin.
Closest hotel is Résidence des îles.
---------------------------

Closest hotel is Original Sokos Hotel Valjus.
-----------------------------
Results for Index 253: murray bridge.
Closest hotel is Balcony On Sixth Lodge.
-----------------------------
Results for Index 254: rocha.
Closest hotel is Hotel Arrarte.
-----------------------------
Results for Index 255: tagusao.
Closest hotel is Caycay future beach resort.
-----------------------------
Results for Index 256: cobija.
Closest hotel is Hotel El Cristo.
-----------------------------
Results for Index 257: port-gentil.
Closest hotel is Fengshui Residence.
-----------------------------
Results for Index 258: te anau.
Closest hotel is Kingsgate Hotel Te Anau.
-----------------------------
Results for Index 259: hami.
Closest hotel is Post And Telecommunication Hotel.
-----------------------------
Results for Index 260: pahrump.
Closest hotel is Best Western Pahrump Oasis.
-----------------------------
Results for Index 261: fort frances.
Closest hotel is LA PLACE RENDEZ-VOUS.
---------------------

Closest hotel is Flia. Ríos Ferreira.
-----------------------------
Results for Index 335: hulan ergi.
Closest hotel is Qiqihaer Zhongtian Hotel.
-----------------------------
Results for Index 336: thayetmyo.
Closest hotel is Ngwe Sin Kyal Guest House.
-----------------------------
Results for Index 337: acapulco.
Closest hotel is HS Hotsson Smart Acapulco.
-----------------------------
Results for Index 338: port shepstone.
Closest hotel is My Den Beachfront Bed and Breakfast | Self-Catering.
-----------------------------
Results for Index 339: tautira.
Closest hotel is Pension Oaoa.
-----------------------------
Results for Index 340: la libertad.
Closest hotel is Hotel La cuesta de Cayma.
-----------------------------
Results for Index 341: raymond.
Closest hotel is Aleteo Catering.
-----------------------------
Results for Index 342: road town.
Closest hotel is ZINGARA 76ft Catamaran, full crew-all included.
-----------------------------
Results for Index 343: yar-sale.
Closest ho

Closest hotel is Bayaz Han (Beyaz Han).
-----------------------------
Results for Index 417: tabou.
Closest hotel is Hotel Les Rochers.
-----------------------------
Results for Index 418: ahipara.
Closest hotel is Ahipara Holiday Park.
-----------------------------
Results for Index 419: byron bay.
Closest hotel is Backpackers Inn.
-----------------------------
Results for Index 420: isangel.
Closest hotel is Tanna Lodge.
-----------------------------
Results for Index 421: saint-joseph.
Closest hotel is Hôtel Château de la Tour.
-----------------------------
Results for Index 422: susuman.
Closest hotel is Susumanzoloto.
-----------------------------
Results for Index 423: plattling.
Closest hotel is Hotel Zum Grünen Baum Plattling.
-----------------------------
Results for Index 424: arenapolis.
Closest hotel is Mato Grosso Hotel.
-----------------------------
Results for Index 425: tocopilla.
Closest hotel is Hotel Galvarino.
-----------------------------
Results for Index 426: arr

Closest hotel is В.Ганбат.
-----------------------------
Results for Index 501: delemont.
Closest hotel is Hotel ibis Delemont Delsberg.
-----------------------------
Results for Index 502: chauk.
Closest hotel is Bo Khin Guest House.
-----------------------------
Results for Index 503: kambove.
Closest hotel is Couvant des soeurs catholiques.
-----------------------------
Results for Index 504: tutoia.
Closest hotel is Pousada Guará's.
-----------------------------
Results for Index 505: touros.
Closest hotel is INN NEW HORIZON.
-----------------------------
Results for Index 506: babati.
Closest hotel is White Rose Lodge.
-----------------------------
Results for Index 507: graaff-reinet.
Closest hotel is Rietjiesbos Self Catering.
-----------------------------
Results for Index 508: noumea.
Closest hotel is Hôtel Le Lagon.
-----------------------------
Results for Index 509: pilot butte.
Closest hotel is Sleep Inn Regina East.
-----------------------------
Results for Index 510: ata

In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 100, 0, 0.2)',
    stroke_color='rgba(0, 0, 100, 0.2)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))